In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 1. Замена лица на видео
from google.colab import drive
import os
import time
import hashlib
import sys

#@markdown   Использовать гугл диск:
use_gdrive = True #@param {type:"boolean"}
#@markdown   Укажите путь до файла изображения с новым лицом:
use_this_face = "" #@param {type:"string"}
#@markdown   Укажите путь до файла с видео или фото, на котором будем делать замену:
replace_faces_in_this_file = "" #@param {type:"string"}
#@markdown   Использовать улучшение лица CodeFormer:
use_codeformer = True #@param {type:"boolean"}
#@markdown   Сбалансируйте влияние улучшения в codeformer: качество (меньшее число) и точность (большее число)
codeformer_fidelity = 0.7 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown   Увеличение выходнго изображения (Метод UpScale от CodeFormer модель RealESRGAN_x2plus)
codeformer_realesrgan_upscale = 1 #@param {type:"number"}
#@markdown   Количество пропускаемых кадров для последующей интерполяции устраняющей флики генеративности
frame_skip = 3 #@param {type:"number"}
#@markdown   Количество запусков рекурсивной интерполяции средней точки (Для каждой входной пары кадров количество выходных кадров будет 2^times_to_interpolate+1)
recursion_times = 2 #@param {type:"slider", min:1, max:4, step:1}
#@markdown   Количество потоков GPU
gpu_threads = 8 #@param {type:"slider", min:1, max:10, step:1}
#@markdown   Сохранить папку с изображениями раскадровки:
keep_frames_directory = False #@param {type:"boolean"}
#@markdown   Поменять лица у всех на видео
all_faces = False #@param {type:"boolean"}
#@markdown  Чистая установка (переустановка удалит content/roop)
wipe__content_roop = False #@param {type:"boolean"}
#@markdown   Пробелы и скобки недопустимые символы в именах файлов <br> Результат будет в той же папке что и изначальное видео/фото с префиксом swapped-

_kf_dir = ""
_all_faces = ""
_face = f'-f={use_this_face}'
_codeformer = ""
_codeformer_fidelity = ""
_codeformer_realesrgan_upscale = ""
_frame_skip = ""
_recursion_times = ""
_gpu_threads = ""

if wipe__content_roop:
    if os.path.exists('/content/roop'):
        !rm -r /content/roop
        !rm -r /content/CodeFormer

if use_gdrive:
    drive.mount('/content/drive')

if keep_frames_directory:
    _kf_dir = "--keep-frames"
else:
    _kf_dir = ""

if all_faces:
    _all_faces = "--all-faces"
else:
    _all_faces = ""

if gpu_threads:
    _gpu_threads = f'--gpu-threads={gpu_threads}'
else:
    _gpu_threads = ""

def is_img(path):
    return path.lower().endswith(("png", "jpg", "jpeg", "bmp"))


if use_codeformer:
    _codeformer = "--codeformer"
    _codeformer_fidelity = f'--codeformer-fidelity={codeformer_fidelity}'
    _codeformer_realesrgan_upscale = f'--codeformer-realesrgan-upscale={codeformer_realesrgan_upscale}'
    _frame_skip = f'--frame-skip={frame_skip}'
    _recursion_times = f'--times_to_interpolate={recursion_times}'

    if not os.path.exists('/content/CodeFormer/inference_codeformer.py'):
        # Clone CodeFormer and enter the CodeFormer folder
        %cd /content
        !rm -rf /content/CodeFormer
        !git clone https://github.com/sczhou/CodeFormer.git /content/CodeFormer
        %cd /content/CodeFormer

        # Set up the environment
        # Install python dependencies
        !pip install -q -r /content/CodeFormer/requirements.txt
        # Install basicsr
        !python /content/CodeFormer/basicsr/setup.py develop

        # Download the pre-trained model
        !python /content/CodeFormer/scripts/download_pretrained_models.py facelib
        !python /content/CodeFormer/scripts/download_pretrained_models.py CodeFormer
        
        !rm -rf /content/CodeFormer/inference_codeformer.py
        !wget -q -O /content/CodeFormer/inference_codeformer.py 'https://raw.githubusercontent.com/operatortob/face-swap-lt-kext/master/assets/CodeFormer/inference_codeformer.py'
        if not is_img(replace_faces_in_this_file):
            %cd /content
            !rm -rf /content/frame-interpolation
            !git clone https://github.com/google-research/frame-interpolation /content/frame-interpolation
            %cd /content/frame-interpolation
            !pip install -r /content/frame-interpolation/requirements.txt
            !pip install apache_beam mediapy
            !wget 'https://huggingface.co/optobsafetens/inswapper_128/resolve/main/pretrained_models-20220214T214839Z-001.zip' -O pretrained_models-20220214T214839Z-001.zip
            !unzip -o "pretrained_models-20220214T214839Z-001.zip"
            !rm -rf pretrained_models-20220214T214839Z-001.zip

            !rm -rf /content/frame-interpolation/eval/interpolator_cli.py
            !wget -q -O /content/frame-interpolation/eval/interpolator_cli.py 'https://raw.githubusercontent.com/operatortob/face-swap-lt-kext/master/assets/frame-interpolation/interpolator_cli.py'

else:
    _codeformer = ""
    _codeformer_fidelity = ""
    _codeformer_realesrgan_upscale = ""
    _frame_skip = ""
    _recursion_times = ""
   
if not os.path.exists('/content/roop/run.py'):

    !apt-get install libgoogle-perftools-dev
    !ln -s libtcmalloc.so.4 libtcmalloc.so
    os.environ.setdefault('LD_PRELOAD', '/usr/lib/x86_64-linux-gnu/libtcmalloc.so')
    os.environ.setdefault('TF_CPP_MIN_LOG_LEVEL', '2')
    os.environ.setdefault('ACCELERATE', 'True')
    os.environ.setdefault('FORCE_CUDA', '1')
    os.environ.setdefault('ATTN_PRECISION', 'fp16')
    os.environ.setdefault('PYTORCH_CUDA_ALLOC_CONF', 'garbage_collection_threshold:0.9,max_split_size_mb:512')   # ещё вариант max_split_size_mb:128
    os.environ.setdefault('CUDA_LAUNCH_BLOCKING', '0')
    os.environ.setdefault('CUDA_CACHE_DISABLE', '0')
    os.environ.setdefault('CUDA_AUTO_BOOST', '1')
    os.environ.setdefault('CUDA_MODULE_LOADING', 'LAZY')
    os.environ.setdefault('CUDA_DEVICE_DEFAULT_PERSISTING_L2_CACHE_PERCENTAGE_LIMIT', '0')
    os.environ.setdefault('GRADIO_ANALYTICS_ENABLED', 'False')
    os.environ.setdefault('SAFETENSORS_FAST_GPU', '1')
    os.environ.setdefault('NUMEXPR_MAX_THREADS', '16')
    os.environ.setdefault('PYTHONHTTPSVERIFY', '0')
    os.environ.setdefault('HF_HUB_DISABLE_TELEMETRY', '1')
    os.environ.setdefault('UVICORN_TIMEOUT_KEEP_ALIVE', '60')
    
    %cd /content

    if os.path.exists('/content/roop'):
        !mv /content/roop /content/roop-temp

    !git clone https://github.com/operatortob/face-swap-lt-kext /content/roop
    
    if os.path.exists('/content/roop-temp'):
        !cp -rf /content/roop-temp/* -t /content/roop && rm -r /content/roop-temp
    
    if not os.path.exists('/content/roop/inswapper_128.onnx'):
        if os.path.exists('/content/drive/MyDrive/roop-data/model/inswapper_128.onnx'):
            !cp -rf /content/drive/MyDrive/roop-data/model/inswapper_128.onnx -t /content/roop
        else:
            !wget -O /content/roop/inswapper_128.onnx 'https://civitai.com/api/download/models/85159?type=Model&format=Other&size=full&fp=fp32'
    
        filename = '/content/roop/inswapper_128.onnx'  # Путь к файлу
        expected_hash = 'e4a3f08c753cb72d04e10aa0f7dbe3deebbf39567d4ead6dce08e98aa49e16af'  # Ожидаемое значение хеш-суммы

        def calculate_hash(file_path):
            # Создание объекта хеша SHA-256
            sha256_hash = hashlib.sha256()

            # Открытие файла в бинарном режиме
            with open(file_path, 'rb') as file:
                # Чтение файла блоками и обновление хеша
                for chunk in iter(lambda: file.read(4096), b''):
                    sha256_hash.update(chunk)

            # Получение окончательного значения хеша
            return sha256_hash.hexdigest()

        # Вычисление хеш-суммы файла
        file_hash = calculate_hash(filename)

        # Сравнение хеш-суммы с ожидаемым значением
        if file_hash == expected_hash:
            print("Хеш-сумма файла модели совпадает.")
        else:
            print("Хеш-сумма файла модели не совпадает. Пробуем скачать ещё раз c huggingface...")
            !rm -r /content/roop/inswapper_128.onnx
            !wget -O /content/roop/inswapper_128.onnx 'https://huggingface.co/optobsafetens/inswapper_128/resolve/main/roopVideoFace_v10.onnx'
            file_hash = calculate_hash('/content/roop/inswapper_128.onnx')
            if file_hash == expected_hash:
                print("Хеш-сумма файла модели совпадает.")
            else:
                print("Хеш-сумма файла модели не совпадает. Без модели ничего не получится! Попробуйте загрузить модель вручную!")
                sys.exit()


if not use_this_face == "" and not replace_faces_in_this_file == "":
    if os.path.exists(f'{replace_faces_in_this_file}'):
        filename_replace_faces_in_this_file = os.path.basename(f'{replace_faces_in_this_file}')
        roop_input_data_target = f'/content/roop-data/{filename_replace_faces_in_this_file}'
        _target = f'-t={roop_input_data_target}'
        if not os.path.exists('/content/roop-data'):
            !mkdir -p /content/roop-data
        !cp -rf $replace_faces_in_this_file $roop_input_data_target

    %cd /content/roop
    !pip install -q -r requirements.txt
    !python3 run.py $_face $_target $_kf_dir $_all_faces $_codeformer $_frame_skip $_recursion_times $_codeformer_fidelity $_codeformer_realesrgan_upscale --gpu-vendor nvidia --keep-fps $_gpu_threads

    if use_gdrive:
        if not os.path.exists('/content/drive/MyDrive/roop-data'):
            !mkdir -p /content/drive/MyDrive/roop-data
        if not os.path.exists('/content/drive/MyDrive/roop-data/model'):
            !mkdir -p /content/drive/MyDrive/roop-data/model
        time.sleep(5)
        !rm -r $roop_input_data_target
        if not os.path.exists('/content/drive/MyDrive/roop-data/model/inswapper_128.onnx'):
            !cp -rf /content/roop/inswapper_128.onnx -t /content/drive/MyDrive/roop-data/model
        !cp -rf /content/roop-data/* -t /content/drive/MyDrive/roop-data
else:
    print('Поля use_this_face и replace_this_face обязательны!')